In [1]:
# !pip install transformers numpy pandas datasets scipy evaluate sklearn
# !pip install scikit-learn
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !pip install accelerate -U

In [3]:
import pandas as pd
import numpy as np
import transformers
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig, TrainingArguments
from scipy.special import softmax
import torch
from transformers import DataCollatorWithPadding
import evaluate
from sklearn.metrics import accuracy_score

In [5]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

True
1
cuda


In [6]:
imdb = load_dataset('imdb')
train,test = pd.DataFrame(imdb['train']), pd.DataFrame(imdb['test']) # not needed

In [5]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest" # can choose some other package from Huggingface instead
MODEL = f"bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
# config = AutoConfig.from_pretrained(MODEL)
# load pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def preprocess_function(examples):
  '''
  truncate sequences to match the maximum input length
  '''
  return tokenizer(examples["text"], truncation=True)
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

NameError: name 'tokenizer' is not defined

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

NameError: name 'tokenizer' is not defined

### Untrained Model
Accuracy
https://huggingface.co/docs/transformers/tasks/sequence_classification

In [8]:
# maybe need a GPU running for all test failes, or batch it properly

# preds = []
# batch_size = 100
# n = len(test)//batch_size
# n = 2
# for i in range(n+1):
#   # if (i+1) % (n // 10) == 0:
#   #   print(f'{round((i+1)*100/n,0)}% complete...')
#   text = list(test.iloc[i*batch_size:(i+1)*batch_size,0])
#   encoded_input = tokenizer(list(text), return_tensors='pt', padding=True, truncation=True) # need to give padding and truncation as true
#   with torch.no_grad():
#     output = model(**encoded_input)
#   preds.append(output.logits.detach().argmax(axis=1))

Looping through each data-point not the best solution// batch it and use GPU instead

In [9]:
# preds = []
# n = len(test)
# for i in range(n):
#   if (i+1) % (n // 10) == 0:
#     print(f'{round((i+1)*100/n,0)}% complete...')
#   text = test.iloc[i,0]
#   encoded_input = tokenizer(str(text), return_tensors='pt', padding=True, truncation=True) # need to give padding and truncation as true
#   with torch.no_grad():
#     output = model(**encoded_input)
#   preds.append(output[0][0].detach().numpy().argmax())

In [10]:
# accuracy_score(test.label, preds)

Finetuning model

In [11]:
from huggingface_hub import notebook_login

# !pip install ipywidgets
notebook_login()

In [12]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [13]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", push_to_hub=True) # default params 
# possible args
# training_args = TrainingArguments(
#     output_dir=output_dir,
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=2,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     push_to_hub=True,
# )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/9375 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


TrainOutput(global_step=9375, training_loss=0.19898288045247395, metrics={'train_runtime': 25886.29, 'train_samples_per_second': 2.897, 'train_steps_per_second': 0.362, 'train_loss': 0.19898288045247395, 'epoch': 3.0})

 inference on fine-tuned model

In [9]:
MODEL = f"sshourie/test_trainer"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
# config = AutoConfig.from_pretrained(MODEL)
# load pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [13]:
preds = []
batch_size = 100
n = len(test)//batch_size
# n = 2
for i in range(n+1):
  if (i+1) % (n // 10) == 0:
    print(f'{round((i+1)*100/n,0)}% complete...')
  text = list(test.iloc[i*batch_size:(i+1)*batch_size,0])
  if len(text) == 0:
    break
  encoded_input = tokenizer(list(text), return_tensors='pt', padding=True, truncation=True) # need to give padding and truncation as true
  with torch.no_grad():
    output = model(**encoded_input)
  preds.append(output.logits.detach().argmax(axis=1))

10.0% complete...
20.0% complete...
30.0% complete...
40.0% complete...
50.0% complete...
60.0% complete...
70.0% complete...
80.0% complete...
90.0% complete...
100.0% complete...


IndexError: list index out of range

In [19]:
print(accuracy_score(test.label, torch.concat(preds)))

0.93648


In [ ]:
print(accuracy_score(test.label, preds))